In [1]:
%matplotlib inline
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
path="F:/AMM2/Magnets/magnet_train_data/magnets_train_14_12_2017/magnets_16_12_2017_tested ok for 165/"
X=[]
y=[]
for directory in os.listdir(path):
    for file in os.listdir(path+directory):
        img=Image.open(path+directory+"/"+file)
        #print(path+directory+"/"+file, img)
        width, height = img.size
        for i in range(width):
            for j in range(height):
                #RGB =img.getpixel((i,j))
                #print(rgb)
                featurevector= img.getpixel((i,j))
                #print(featurevector)
        X.append(featurevector)
        if directory == "T1":
            label = 0
        if directory == "pore":
            label = 1
        if directory == "oxide":
            label = 2
        if directory == "Nd":
            label = 3
        if directory == "T2":
            label = 4
        if directory == "mixed_phases":
            label = 5
        if directory == "UIO":
            label = 6
        y.append(label)      

In [2]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
validation_size = 0.33
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,test_size=validation_size, random_state= 7)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [4]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape)

(64L, 3L)


In [5]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(activation='relu', alpha=0.01, batch_size='auto',hidden_layer_sizes=(20,20),max_iter=600)
nn.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300, 300), learning_rate='constant',
       learning_rate_init=0.001, max_iter=600, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [6]:
predictions = nn.predict(X_test)

In [7]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[6 0 0 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 6 1 0 0 0]
 [0 0 1 6 0 0 0]
 [0 0 0 0 3 0 0]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 5]]


In [8]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         6
          1       1.00      1.00      1.00         2
          2       0.86      0.86      0.86         7
          3       0.86      0.86      0.86         7
          4       1.00      1.00      1.00         3
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00         5

avg / total       0.94      0.94      0.94        32



In [30]:
def relative_phase(img):
    white=0
    black=0
    brown=0
    other1=0
    other2=0
    other3=0
    other4=0
    for pixel in img.getdata():
        if pixel == (0,255,0):
            white += 1
        elif pixel == (255,255,0):
            brown += 1
        elif pixel == (0,255,255):
            black += 1
        elif pixel == (255,0,255):
            other1 += 1
        elif pixel == (0,0,255):
            other2 += 1
        elif pixel == (255,127,0):
            other4 += 1
        else:
            other3 += 1
    all = width*height
    return ((100.0*white/all),(100.0*brown/all),(100.0*black/all),(100.0*other1/all),(100.0*other2/all),(100.0*other3/all),(100.0*other4/all))

In [32]:
%matplotlib inline
from PIL import ImageFilter
from PIL import Image, ImageEnhance
path1 = "C:/test/"
#path1="F:/Intellesis/magnets/my results/resized/"
classcolors = {5 : (255,127,0), 0 : (0,255,0), 1 : (0,0,255), 4 : (255,255,0), 3 : (0,255,255), 2 : (255,0,255), 6 : (255,0,0)}
import numpy as np
for file in os.listdir(path1):
        #timer = Timer()
        #timer.start()
        #print("Loading image... at " + timer.elapsed(""))
        print(path1+directory+"/"+file)
        img=Image.open(path1+"/"+file)
        width, height = img.size
        #img = im5.filter(ImageFilter.UnsharpMask(radius=3, percent=200, threshold=1))
        #img.save("C:/3/001.png")
        flatImg = np.array(img).reshape(-1,3)
        #predict
        #print("\t\t\Predicting... at " + timer.elapsed(""))
        predictions = nn.predict(scaler.transform(flatImg))
        #color image for visualization
        #print("\t\t\t\Coloring... at " + timer.elapsed(""))
        i=0
        for pred in predictions:
            flatImg[i] = classcolors[int(pred)]
            i=i+1
        #print("\t\t\t\t\openflatten...at " + timer.elapsed(""))
        flatImg = flatImg.reshape(height, width,3)
        img = Image.fromarray(flatImg.astype('uint8'), 'RGB')
        
        #print("___Done... at " + timer.elapsed(""))
        
        ee = relative_phase(img)
        print(ee)
img.save("C:/segmented/testts.png")
plt.imshow(img)

C:/test/UIO/Result of 165_1.jpg
(85.64413956931128, 1.2992959564835695, 4.611915377067805, 4.445544213588557, 1.6046683765686056, 1.1447162308711893, 1.2497202761089976)
